In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils import data
import matplotlib.gridspec as gridspec

In [ ]:
tau = 1
number_subsystems = 10
fake_dims = 10
output_sizes = [2 for _ in range(number_subsystems)]
# tau list for timescales estimation
msmlags = np.arange(1, 10)

# Batch size for Stochastic Gradient descent
batch_size = 10000

# How many hidden layers the network chi has
network_depth = 3

# Width of every layer of chi
layer_width = 30

# Learning rate used for the ADAM optimizer

# create a list with the number of nodes for each layer
nodes = [layer_width]*network_depth

# Definition of the hidden Markov transition matrices
eps_list = np.linspace(0.,.1, number_subsystems+1)[1:]
lam = 0 #0.04
# Number of unformative noise dimensions
dim_noise = 10

# How strong the fake subsystem is
factor_fake = 3.
# How large the noise in the mask for regularization is
noise = 2.
# Threshold after which the attention weight is set to zero
cutoff=0.9

# Learning rate
learning_rate=0.005
# epsilon for inversion of symmetric matrices
epsilon=1e-8
# score method
score_mode='regularize' # one of ('trunc', 'regularize', 'clamp', 'old')

### Create toymodel

In [ ]:
from examples import HyperCube

In [ ]:
toymodel = HyperCube(eps_list, lam=lam)

### Sample hidden and observable trajectory

In [ ]:
angles = np.pi / 4 * np.ones(number_subsystems//2)
hidden_state_traj, observable_traj = toymodel.generate_traj(100000, angles=angles, dim_noise=dim_noise)
hidden_state_traj_valid, observable_traj_valid = toymodel.generate_traj(10000, angles=angles, dim_noise=dim_noise)

### Define training and validation set

In [ ]:
from deeptime.util.data import TrajectoryDataset

train_data = TrajectoryDataset(lagtime=tau, trajectory=observable_traj.astype('float32'))
val_data = TrajectoryDataset(lagtime=tau, trajectory=observable_traj_valid.astype('float32'))

### Define networks

In [ ]:
from masks import Mask
from collections import OrderedDict
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
input_size = observable_traj.shape[1] 
mask = Mask(input_size, number_subsystems, mean=torch.Tensor(train_data.data.mean(0)),
            std=torch.Tensor(train_data.data.std(0)), factor_fake=factor_fake, noise=noise, 
            device=device, cutoff=cutoff)
mask.to(device=device)
lobes = []
for output_size in output_sizes:
    lobe_dict = OrderedDict([('Layer_input', nn.Linear(input_size, layer_width)),
                            ('Elu_input', nn.ELU())])
    for d in range(network_depth):
        lobe_dict['Layer'+str(d)]=nn.Linear(layer_width, layer_width)
        lobe_dict['Elu'+str(d)]=nn.ELU()
    lobe_dict['Layer_output']=nn.Linear(layer_width, output_size)
    lobe_dict['Softmax']=nn.Softmax(dim=1) # obtain fuzzy probability distribution over output states
    
    lobe = nn.Sequential(
        lobe_dict 
    )
    lobes.append(lobe.to(device=device))

print(mask)
print(lobes)
                 

### Create iVAMPnets estimator

In [ ]:
from ivampnets import iVAMPnet

In [ ]:
ivampnet = iVAMPnet(lobes, mask, device, learning_rate=learning_rate, epsilon=epsilon, score_mode=score_mode)

### Plot mask before training

In [ ]:
from examples import plot_mask
plot_mask(mask, skip=2)

### Create data loader

In [ ]:
from torch.utils.data import DataLoader

loader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(val_data, batch_size=len(val_data), shuffle=False)

### Create a tensorboard writer to observe performance during training

In [ ]:
tensorboard_installed = True
if tensorboard_installed:
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('./runs/Cube10/')
    input_model, _ = next(iter(loader_train))
    # writer.add_graph(lobe, input_to_model=input_model.to(device))
else:
    writer=None

### Fit the model on the training data

In [ ]:
model = ivampnet.fit(loader_train, n_epochs=25, validation_loader=loader_val, mask=True, lam_decomp=2, 
                     lam_trace=0.5, start_mask=0, end_trace=1, tb_writer=writer, clip=False).fetch_model()
mask.noise = 5
model = ivampnet.fit(loader_train, n_epochs=25, validation_loader=loader_val, mask=True, lam_decomp=2, 
                     lam_trace=0, start_mask=0, end_trace=0, tb_writer=writer, clip=False).fetch_model()
mask.noise = 10
model = ivampnet.fit(loader_train, n_epochs=25, validation_loader=loader_val, mask=True, lam_decomp=2, 
                     lam_trace=0, start_mask=0, end_trace=0, tb_writer=writer, clip=False).fetch_model()

### Plot the training and validation scores

In [ ]:
plt.loglog(*ivampnet.train_scores.T, label='training')
plt.loglog(*ivampnet.validation_scores.T, label='validation')
plt.xlabel('step')
plt.ylabel('score')
plt.legend();

### Plot the mask after training

In [ ]:
plot_mask(mask, vmax=0.5, skip=2)

### Estimate implied timescales

In [ ]:
mask.noise=0 # set the noise to zero. 
its = []
for tau_i in msmlags:
    val_data_temp = TrajectoryDataset(lagtime=tau_i, trajectory=observable_traj_valid.astype('float32'))
    its.append(model.timescales(val_data_temp.data, val_data_temp.data_lagged, tau_i))
# Convert to array
its = np.array(its)
# Change the shape
its = np.transpose(its, axes=[1,0,2])
# Estimate the true timescales of the hidden Markov Chain
eigvals_true = np.array(toymodel.eigvals_list_coupled).flatten()
its_true = -1/np.log(eigvals_true)

In [ ]:
from examples import plot_hypercube_its

In [ ]:
plot_hypercube_its(its, msmlags, its_true, ylog=True)